In [13]:
import os
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer,BitsAndBytesConfig
import json
from Gpt4_OpenAI_Prompt import system_msg,context,postprocess

In [17]:
from Newprompt import system_msg as newprompt

In [2]:
print(torch.cuda.is_available())
print(torch.cuda.get_device_name())

True
Tesla T4


In [3]:
################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False
bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)
# Load the entire model on the GPU 0
device_map = {"": 0}


In [4]:
model_name = "NousResearch/llama-2-7b-chat-hf"
model = AutoModelForCausalLM.from_pretrained(model_name,quantization_config=bnb_config,
    device_map=device_map)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
#Testing prompt

# question = "what are the sales in africa"
# prompt = f"""[INST]<<SYS>>
# Assume you are a system that detects the month and year in the given secentance, user will give only provide the input in english.
# For example if user gives "top  orders in march 2020", you should only return march 2020.

# <</SYS>>
# {question}
# [/INST]
# [END]"""

In [9]:
def gereneate_output_llama2(prompt):
    #prompt = json.dumps(prompt)
    model_input = tokenizer(prompt, return_tensors="pt").to("cuda")
    #model.eval()
    response = tokenizer.decode(model.generate(**model_input, max_length=4096)[0], skip_special_tokens=True)
    return response

In [19]:
################################################################################
# Setting the same prompt which is used for GPT4 open AI
################################################################################

def get_user_question_for_open_ai_prompt(question):
    #return "Based on the following context:\n" + str(context) + "\n return a response for the question:\n" + question + "\n"
    return "\n Now return a response for this question:\n" + question + "\n"

def compose_prompt_open_ai_prompt(question,base_prompt=system_msg):
    return f"""[INST]<<SYS>>
            {system_msg}
            <</SYS>>
            {get_user_question_for_open_ai_prompt(question)}
            [/INST]
            [END]"""
def post_process_open_ai_prompt_response(output):
    mql = output.split("[END]")[1].split("\n\nReasoning:\n\n")[0]
    reason = output.split("[END]")[1].split("\n\nReasoning:\n\n")[1]
    print("*"*50+"MQL"+"*"*50)
    print(mql)
    print("\n"+"*"*50+"Reason"+"*"*50)
    print(reason)
    #return mql,reason

In [16]:
%%time
question = "top 5 segments basis discount"
output = gereneate_output_llama2(compose_prompt_open_ai_prompt(question))
post_process_open_ai_prompt_response(output)

**************************************************MQL**************************************************


The response for the question "top 5 segments basis discount" is:

{
"MEASURE": {
"n-gram matched to MEASURE": [
    {
        "ENTITY": "Matched MEASURE",
        "MEASURE CONSTRAINT": [
            {
                "COMPARISON VALUE": "discount",
                "COMPARSION OPERATOR": ">"
            }
        ],
        "ADJECTIVE": [],
        "TONE": "positive"
    }
],
"DIMENSION": {
"n-gram matched to DIMENSION": [
    {
        "ENTITY": "Matched DIMENSION",
        "RANK": [
            {
                "RANK ADJECTIVE": "top",
                "RANK VALUE": "5"
            }
        ],
        "ADJECTIVE": [],
        "TONE": "positive"
    }
],
"FILTER": {
"n-gram matched to FILTER": [
    {
        "ENTITY": "Matched FILTER",
        "PARENT": "parent of the Matched FILTER",
        "EXCLUDE": ""
    }
],
"DERIVED MEASURE": {
"n-gram matched to DERIVED MEASURE": [
    

In [ ]:
%%time
question = "top 5 segments basis discount"
output = gereneate_output_llama2(compose_prompt_open_ai_prompt(question,base_prompy))
post_process_open_ai_prompt_response(output)

In [14]:
%%time
question = "top 5 segments basis discount"
output=gereneate_output_llama2(compose_prompt_open_ai_prompt(question))
post_process_open_ai_prompt_response(output)

**************************************************MQL**************************************************


The response for the question "top 5 segments basis discount" is as follows:

{
"MEASURE": {
"n-gram matched to MEASURE": [
{
"ENTITY": "Matched MEASURE",
"MEASURE CONSTRAINT": [
{
"COMPARISON VALUE": "top 5",
"COMPARISION OPERATOR": "="
}
],
"ADJECTIVE": [],
"TONE": ""
}
],
"DIMENSION": {
"n-gram matched to DIMENSION": [
{
"ENTITY": "Matched DIMENSION",
"RANK": [
{
"RANK ADJECTIVE": "top",
"RANK VALUE": "5"
}
],
"ADJECTIVE": [],
"TONE": ""
}
],
"FILTER": {
"n-gram matched to FILTER": [
{
"ENTITY": "Matched FILTER",
"PARENT": "Segment",
"EXCLUDE": ""
}
],
"DERIVED MEASURE": {
"n-gram matched to DERIVED MEASURE": [
{
"ENTITY": "Matched DERIVED MEASURE",
"RATIO FILTER": [
{}
],
"APPLIED MEASURE": [
{
"n-gram matched to MEASURE": [
{
"ENTITY": "Discount",
"RANK": [
{
"RANK ADJECTIVE": "top",
"RANK VALUE": "5"
}
]
}
],
"ADJECTIVE": [],
"TONE": ""
}
],
"DATE VARIABLE": {
"asked time ele

In [70]:
%%time
question = "discount rate of phone and binders"
output=gereneate_output_llama2(compose_prompt(question))
post_process(output)

**************************************************MQL**************************************************


The response for the question "discount rate of phone and binders" is:

{
"MEASURE": [
{
"ENTITY": "Discount",
"other names": ["discount", "discount rate", "discount value", "deduction"]
},
{
"ENTITY": "Purchase Vol",
"other names": ["purchase", "purchase value", "purchase model"]
},
{
"ENTITY": "Quantity",
"other names": ["quantity", "volume"]
},
{
"ENTITY": "Sales",
"other names": ["sales", "sale"]
}
],
"DIMENSION": [
{
"ENTITY": "Sub-Category",
"other names": ["sub-category", "sub category", "categories", "section"]
},
{
"ENTITY": "Segment",
"other names": ["segment", "segments", "units", "divisions"]
},
{
"ENTITY": "Parts",
"other names": ["parts", "part", "section", "divisions"]
},
{
"ENTITY": "Country",
"other names": ["country", "countries"]
}
],
"FILTER": [
{
"ENTITY": "Consumer",
"other names": ["consumers", "consumer"],
"parent": "Segment"
},
{
"ENTITY": "Phone",
"other n